实现结合了rarank策略的优化，使用rerank model重排序

In [14]:
import nest_asyncio

nest_asyncio.apply()
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.notebook_utils import display_source_node

In [15]:
from readai.components.retrievers import BM25Retriever
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.retrievers import QueryFusionRetriever

In [16]:
# 加载.env文件
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()

# 获取项目根目录路径
project_root = Path(os.getenv("PROJECT_ROOT"))
test_data_path = project_root / "readai/tests/data"

In [17]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.deepseek import DeepSeek

api_key = os.getenv("DEEPSEEK_API_KEY")
model_name = os.getenv("DEEPSEEK_MODEL")
embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5", base_url="http://localhost:11434"
)

# 设置 LLM
llm = DeepSeek(model=model_name, api_key=api_key)

In [18]:
from llama_index.core.indices import load_index_from_storage
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import AsyncQdrantClient, QdrantClient

client = QdrantClient(url="http://localhost:6333")
aclient = AsyncQdrantClient(url="http://localhost:6333")

加载本地indexstroe,docstore，还有向量库数据

In [19]:
collection_name = "hybrid_nodes"
persist_dir = test_data_path / collection_name

In [20]:
vector_store_base = QdrantVectorStore(
    client=client,
    aclient=aclient,
    collection_name=collection_name,
)

In [21]:
hybrid_index = None
if client.collection_exists(collection_name):
    print(f"collection_name: '{collection_name}' 已存在")
    # 从本地加载storage_context
    storage_context = StorageContext.from_defaults(
        persist_dir=persist_dir, vector_store=vector_store_base
    )
    hybrid_index = load_index_from_storage(
        storage_context, llm=llm, embed_model=embed_model
    )
    print("加载向量完成")
else:
    print(f"collection_name: '{collection_name}' 不存在")

collection_name: 'hybrid_nodes' 已存在
加载向量完成


In [22]:
# 获取docs字典
# hybrid_all_nodes_dict = hybrid_index.storage_context.docstore.docs

创建检索器

In [23]:
import jieba

stop_words_path = project_root / "readai/utils/baidu_stopwords.txt"
custom_tokenizer = jieba.Tokenizer()
custom_tokenizer.load_userdict(str(stop_words_path))
# 加载stop_words为字符串列表
with open(stop_words_path) as f:
    stop_words = f.readlines()

stop_words = set(stop_words)
print(len(stop_words))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/70/hwlr8qxx6g751fyxb33mcys40000gn/T/jieba.cache
Loading model cost 0.265 seconds.
Prefix dict has been built successfully.


1396


In [60]:
bm25_retriever = BM25Retriever.from_defaults(
    index=hybrid_index,
    similarity_top_k=5,
    tokenizer=custom_tokenizer,
    stopwords=stop_words,
)

In [62]:
vector_retriever = hybrid_index.as_retriever(similarity_top_k=5)
small2big_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=hybrid_index.docstore.docs,
    verbose=True,
)

In [26]:
# 获取向量召回结果
# dense_nodes = small2big_retriever.retrieve(
#     "在日常交流中，我们如何区分客观的观察和主观的评价？"
# )

In [27]:
# print("召回数量", len(dense_nodes))
# for chunk_node in dense_nodes:
#     display_source_node(chunk_node, source_length=1500)


Hybrid 检索

In [28]:
from enum import Enum


class FUSION_MODES(str, Enum):
    """Enum for different fusion modes."""

    RECIPROCAL_RANK = "reciprocal_rerank"  # apply reciprocal rank fusion
    RELATIVE_SCORE = "relative_score"  # apply relative score fusion
    DIST_BASED_SCORE = "dist_based_score"  # apply distance-based score fusion
    SIMPLE = "simple"  # simple re-ordering of results based on original scores

In [69]:
QUERY_GEN_PROMPT_ZH = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries in Chinese, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)

In [70]:
fusion_retriever = QueryFusionRetriever(
    retrievers=[small2big_retriever, bm25_retriever],
    # retriever_weights=[0.5, 0.5],
    mode=FUSION_MODES.RECIPROCAL_RANK,
    verbose=True,
    similarity_top_k=10,
    llm=llm,
    num_queries=2,  # 不需要查询扩展,查询重写
    query_gen_prompt=QUERY_GEN_PROMPT_ZH,
)


Evaluation

In [30]:
from llama_index.core.evaluation import (
    EmbeddingQAFinetuneDataset,
    RetrieverEvaluator,
)

# 从本地加载测试集
eval_dataset_from_hybrid_nodes_50 = EmbeddingQAFinetuneDataset.from_json(
    "test_qa_datasets.json",
)

In [71]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics,
    retriever=fusion_retriever,
    # node_postprocessors=my_node_postprocessor # 这里可以用rerank集成
)
eval_results = await retriever_evaluator.aevaluate_dataset(
    eval_dataset_from_hybrid_nodes_50,
    show_progress=True,
)

Generated queries:
1. 第一对夫妻如何成功挽救婚姻避免离婚？
Generated queries:
1. 第二对夫妻最初是如何表达彼此的需求的？
Retrieving with query id None: 第一对夫妻通过什么方式避免了离婚？


2025-04-09 15:46:14.110 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 15:46:14.190 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 15:46:14.234 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 15:46:14.282 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: 69ae5f8e-3282-4aff-ab27-c70f849acead
Retrieving with query id 69ae5f8e-3282-4aff-ab27-c70f849acead: 第一对夫妻通过什么方式避免了离婚？
Retrieved node with id, entering: f2932829-0d06-48d4-bdd6-05309f04664a
Retrieving with query id f2932829-0d06-48d4-bdd6-05309f04664a: 第一对夫妻通过什么方式避免了离婚？
Retrieved node with id, entering: a2f5016c-6ee4-4cbb-add4-7292ea02877d
Retrieving with query id a2f5016c-6ee4-4cbb-add4-7292ea02877d: 第一对夫妻通过什么方式避免了离婚？
Retrieving with query id None: 1. 第一对夫妻如何成功挽救婚姻避免离婚？
Retrieved node with id, entering: 69ae5f8e-3282-4aff-ab27-c70f849acead
Retrieving with query id 69ae5f8e-3282-4aff-ab27-c70f849acead: 1. 第一对夫妻如何成功挽救婚姻避免离婚？
Retrieved node with id, entering: a2f5016c-6ee4-4cbb-add4-7292ea02877d
Retrieving with query id a2f5016c-6ee4-4cbb-add4-7292ea02877d: 1. 第一对夫妻如何成功挽救婚姻避免离婚？
Retrieving with query id None: 第二对夫妻最初如何表达“需要”？
Retrieved node with id, entering: a2f5016c-6ee4-4cbb-add4-7292ea02877d
Retrieving with query id a2f5016c-6ee4-4cbb-add4-7292ea02877

2025-04-09 15:46:25.149 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 15:46:25.199 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 15:46:25.251 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Generated queries:
1. 愤怒的心理学根源有哪些？
2. 愤怒情绪产生的生理机制是什么？
3. 如何从社会学角度理解愤怒的成因？
4. 愤怒与童年经历的关系研究
5. 文化差异对愤怒表达的影响
Retrieving with query id None: 非暴力沟通如何看待愤怒？
Retrieved node with id, entering: a8f63fdd-3893-4761-be42-114957027a8b
Retrieving with query id a8f63fdd-3893-4761-be42-114957027a8b: 非暴力沟通如何看待愤怒？
Retrieved node with id, entering: 5508cb82-2abe-44f0-b9f1-72c5202c14d2
Retrieving with query id 5508cb82-2abe-44f0-b9f1-72c5202c14d2: 非暴力沟通如何看待愤怒？
Retrieving with query id None: 1. 非暴力沟通中愤怒情绪的处理方法有哪些？
Retrieved node with id, entering: 5508cb82-2abe-44f0-b9f1-72c5202c14d2
Retrieving with query id 5508cb82-2abe-44f0-b9f1-72c5202c14d2: 1. 非暴力沟通中愤怒情绪的处理方法有哪些？
Retrieved node with id, entering: 794e0320-1392-4339-a80c-009435c45f6d
Retrieving with query id 794e0320-1392-4339-a80c-009435c45f6d: 1. 非暴力沟通中愤怒情绪的处理方法有哪些？
Retrieving with query id None: 愤怒的根本原因是什么？
Retrieved node with id, entering: a8f63fdd-3893-4761-be42-114957027a8b
Retrieving with query id a8f63fdd-3893-4761-be42-114957027a8b: 愤怒的根本原因是什么？

...

Generated queries:
1. 同理心的心理学定义是什么？
Retrieving with query id None: "非暴力呐喊"是什么？
Retrieved node with id, entering: 7c9e6c07-4920-4602-8d30-2b77861b4991
Retrieving with query id 7c9e6c07-4920-4602-8d30-2b77861b4991: "非暴力呐喊"是什么？
Retrieved node with id, entering: 5f44342d-77d1-4243-a602-6b0ecbfe5edf
Retrieving with query id 5f44342d-77d1-4243-a602-6b0ecbfe5edf: "非暴力呐喊"是什么？
Retrieved node with id, entering: ecf04b61-fd74-4e7d-a44a-5ac13a8d5003
Retrieving with query id ecf04b61-fd74-4e7d-a44a-5ac13a8d5003: "非暴力呐喊"是什么？
Retrieving with query id None: 1. "非暴力呐喊" 的具体含义和起源是什么？
Retrieved node with id, entering: 7c9e6c07-4920-4602-8d30-2b77861b4991
Retrieving with query id 7c9e6c07-4920-4602-8d30-2b77861b4991: 1. "非暴力呐喊" 的具体含义和起源是什么？
Retrieved node with id, entering: 5f44342d-77d1-4243-a602-6b0ecbfe5edf
Retrieving with query id 5f44342d-77d1-4243-a602-6b0ecbfe5edf: 1. "非暴力呐喊" 的具体含义和起源是什么？
Retrieved node with id, entering: 9e1944ff-7cc3-4ab2-b858-9227905425c2
Retrieving with query id 9e1944ff-7cc3-4a

2025-04-09 15:53:57.490 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 15:53:57.545 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...
2025-04-09 15:53:57.588 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Generated queries:
1. 董事长转开脸的原因是什么？
Retrieving with query id None: 谈话中多少人看向了说话者？
Retrieved node with id, entering: ccafb2d8-78ab-4f16-8f0e-5f368871b850
Retrieving with query id ccafb2d8-78ab-4f16-8f0e-5f368871b850: 谈话中多少人看向了说话者？
Retrieved node with id, entering: 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e
Retrieving with query id 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e: 谈话中多少人看向了说话者？
Retrieved node with id, entering: af90f2ee-62d7-4fbc-b60d-28cd2a88327d
Retrieving with query id af90f2ee-62d7-4fbc-b60d-28cd2a88327d: 谈话中多少人看向了说话者？
Retrieved node with id, entering: c327e5b0-320f-4e6c-a67d-609d6b16da68
Retrieving with query id c327e5b0-320f-4e6c-a67d-609d6b16da68: 谈话中多少人看向了说话者？
Retrieving with query id None: 1. 在对话中，有多少人会注视说话者？
Retrieved node with id, entering: 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e
Retrieving with query id 915cb2be-ebb3-4aaf-93d3-c6e5c062b80e: 1. 在对话中，有多少人会注视说话者？
Retrieved node with id, entering: 678f652b-5d93-443f-ac4a-08a0ef80292e
Retrieving with query id 678f652b-5d93-443f-ac4a-08

2025-04-09 15:53:57.626 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: ccafb2d8-78ab-4f16-8f0e-5f368871b850
Retrieving with query id ccafb2d8-78ab-4f16-8f0e-5f368871b850: 1. 董事长转开脸的原因是什么？
Retrieved node with id, entering: 9465ac7b-bdd4-47bc-817c-18e292b85b0d
Retrieving with query id 9465ac7b-bdd4-47bc-817c-18e292b85b0d: 1. 董事长转开脸的原因是什么？
Retrieved node with id, entering: 3318465e-bb4f-44fd-90a2-c5f5239ed118
Retrieving with query id 3318465e-bb4f-44fd-90a2-c5f5239ed118: 1. 董事长转开脸的原因是什么？
Retrieved node with id, entering: af90f2ee-62d7-4fbc-b60d-28cd2a88327d
Retrieving with query id af90f2ee-62d7-4fbc-b60d-28cd2a88327d: 1. 董事长转开脸的原因是什么？


In [32]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""
    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)
    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

## 对比实验记录

In [25]:
# 2025-04-09下午14:31
# bm25 topk=10;vector topk=10;
# mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=10; weight=[0.3,0.7]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.66,0.286,0.132,0.66,0.286,0.379367


In [36]:
# 2025-04-09下午14:36
# bm25 topk=10;vector topk=10;
# fusion mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=5; weight=[0.3,0.7]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.66,0.286,0.132,0.66,0.286,0.379367


In [42]:
# 2025-04-09下午两点
# bm25 topk=10;vector topk=10;
# fusion mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=5; weight=[0.5,0.5]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.76,0.657667,0.152,0.76,0.657667,0.682369


In [46]:
# 2025-04-09下午两点
# bm25 topk=10;vector topk=10;
# fusion mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=10; weight=[0.5,0.5]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.84,0.66779,0.0855,0.84,0.66779,0.707663


In [33]:
# 2025-04-09下午两点
# bm25 topk=10;vector topk=10;
# fusion mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=10; weight=[0.4,0.6]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.84,0.431425,0.0855,0.84,0.431425,0.530861


In [43]:
# 2025-04-09下午两点
# bm25 topk=5;vector topk=5;
# fusion mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=10; weight=[0.5,0.5]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.82,0.637798,0.131897,0.82,0.637798,0.680123


In [47]:
# 2025-04-09下午两点50分
# bm25 topk=10;vector topk=5;
# fusion mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=10; weight=[0.5,0.5]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.83,0.632135,0.087095,0.83,0.632135,0.676776


In [52]:
# 2025-04-09下午两点53分
# bm25 topk=5;vector topk=10;
# fusion mode=FUSION_MODES.RELATIVE_SCORE; fusion topk=10; weight=[0.5,0.5]
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.86,0.657992,0.111933,0.86,0.657992,0.705287


采取 FUSION_MODES.RELATIVE_SCORE ，最好是权重相同，侧重于基于向量召回更多，减少因为关键字检索带来的干扰

In [57]:
# 采用RRF策略
# 2025-04-09下午两点55分
# bm25 topk=10;vector topk=10;
# fusion mode=FUSION_MODES.RECIPROCAL_RANK; fusion topk=10;
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.56,0.43804,0.057278,0.56,0.43804,0.467542


In [66]:
# 采用RRF策略
# 2025-04-09下午3点
# bm25 topk=5; vector topk=5;
# fusion mode=FUSION_MODES.RECIPROCAL_RANK; fusion topk=10;
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.65,0.538595,0.101528,0.65,0.538595,0.565733


In [72]:
# 2025-04-09下午4点
# bm25 topk=5; vector topk=5;
# num_queries=2; 说明查询重写，有一定优化效果
# fusion mode=FUSION_MODES.RECIPROCAL_RANK; fusion topk=10;
display_results("hybrid_nodes_50", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,hybrid_nodes_50,0.73,0.59644,0.083083,0.73,0.59644,0.629213


判断认为，在对于片段语义理解要求较高的场景，可能还是更倾向于多利用语义信息的召回来提升检索效果，下面考虑结合rerank model进一步提升效果

In [73]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)

In [ ]:
from FlagEmbedding import FlagReranker

reranker = FlagReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)
scores = reranker.compute_score(
    [
        ["what is panda?", "hi"],
        [
            "what is panda?",
            "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
        ],
    ]
)
print(scores)  # [-8.1875, 5.26171875]

In [ ]:
bge_rerank_model = "BAAI/bge-reranker-v2-m3"
bge_rerank = FlagEmbeddingReranker(model=bge_rerank_model, top_n=5)

实现结合BM25,small2big,最后进行rerank的retriever

In [ ]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics,
    retriever=fusion_retriever,
    node_postprocessors=[bge_rerank],
)

eval_results = await retriever_evaluator.aevaluate_dataset(
    eval_dataset_from_hybrid_nodes_50,
    show_progress=True,
)